In [63]:
import sys
import os
import glob
import tensorflow as tf
import numpy as np
sys.path.append("/".join(os.getcwd().split("/")[:-1]))
#tf.InteractiveSession()
tf.enable_eager_execution()

In [64]:
from src.utils.audio import load_processed_audio

In [65]:
positives, negatives, backgrounds = load_processed_audio()

W0829 15:46:44.431561 140734837728704 deprecation_wrapper.py:119] From /Users/az01640/Projets/multrigger-word/src/utils/audio.py:40: The name tf.read_file is deprecated. Please use tf.io.read_file instead.

W0829 15:46:45.682397 140734837728704 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0829 15:46:45.696272 140734837728704 deprecation.py:323] From /Users/az01640/Projets/multrigger-word/src/utils/audio.py:42: decode_audio (from tensorflow.contrib.ffmpeg.ffmpeg_ops) is deprecated and will be removed after 2018-09-04.
Instructions for updating:
tf.contrib.ffmpeg will be removed in 2.0, the support for video and audio will continue to be provided in tens

In [66]:
from src.train import *
from src.make_dataset import *

In [67]:
ds = tf.data.Dataset.from_generator(background_generator(backgrounds, 10*16000, 32), output_types=tf.float32)

W0829 15:47:34.600429 140734837728704 deprecation.py:323] From /Users/az01640/Projets/multrigger-word/.venv/lib/python3.6/site-packages/tensorflow/python/data/ops/dataset_ops.py:494: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


In [68]:
%%timeit
ds = tf.data.Dataset.from_generator(background_generator(backgrounds, 10*16000, 32), output_types=tf.float32)
ds = ds.map(add_word(positives, negatives, .2))
for x in ds:
    pass
x

189 ms ± 8.86 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%%timeit
ds = tf.data.Dataset.from_generator(background_generator(backgrounds, 10*16000, 32), output_types=tf.float32)
ds = ds.map(add_word(positives, negatives, .2))
ds = ds.map(spectrogram_from_samples_batch)
for x in ds:
    pass
x

W0827 11:47:20.064616 140734956590528 deprecation_wrapper.py:119] From /Users/az01640/Projets/multrigger-word/src/make_dataset.py:72: The name tf.log is deprecated. Please use tf.math.log instead.



581 ms ± 18 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
sns.heatmap(np.swapaxes(x[0].numpy()[0], 0, 1))

In [16]:
import os
import time
import numpy as np

import tensorflow as tf
tf.enable_eager_execution()

In [5]:
import random

In [70]:
backgrounds = np.eye(10)
WORDS = list(range(2, 10))

def insert_random(background):

    backs = []

    for back in background:
        word = random.choice(WORDS)
        backs.append(tf.concat([back[:2], [word], back[3:]], axis=0))

    return tf.stack(backs)

ds = tf.data.Dataset.from_tensor_slices(backgrounds).batch(5).map(lambda x: tf.py_function(insert_random, [x], 'float64'))

for record in ds:
    print(record)

tf.Tensor(
[[1. 0. 2. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 4. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 8. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 2. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 3. 0. 1. 0. 0. 0. 0. 0.]], shape=(5, 10), dtype=float64)
tf.Tensor(
[[0. 0. 9. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 5. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 3. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 9. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 9. 0. 0. 0. 0. 0. 0. 1.]], shape=(5, 10), dtype=float64)
